In [2]:
import random
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [3]:
data = pd.read_csv('caesarian.csv')
non_categorical=['Age']
decision_column_name='Caesarian'
count_of_yes_no=[]
tables={}

In [4]:
#All function
def extract_feature():
  return list(train.columns)[:-1]
def find_unique(feature_column):
  return sorted(feature_column.unique())
def filter_column(data,column,filter_value):
  return data[column==filter_value]
def prior_probablity_calculation(decision_column):
  classes=find_unique(decision_column)
  prior_probablity=[]
  for i in classes:
    filtered_data=filter_column(train,decision_column,i)
    count_of_yes_no.append(len(filtered_data))
    prior_probablity.append(len(filtered_data)/len(train))
  return prior_probablity
def is_feature_categorical(feature_name):
  for i in non_categorical:
    if(feature_name==i):
      return False
  return True

def table(feature_column,decision_column):
  unique_attributes=find_unique(feature_column)
  decision_values=find_unique(decision_column)
  feat_table={}
  for i in unique_attributes:
    probablity_feature_data_with_decision=[]
    feature_data=filter_column(train,feature_column,i)
    for j in decision_values:
      feature_data_with_decision=filter_column(feature_data,feature_data[decision_column_name],j)  
      probablity_feature_data_with_decision.append(len(feature_data_with_decision)/count_of_yes_no[j])
    probablity_feature_data_with_decision.append(len(feature_data))
    feat_table[i]=probablity_feature_data_with_decision
  return feat_table

def calculate_probablity(feat_name,feat_value,decision_value,decision_column):
  d=filter_column(train,decision_column,decision_value)
  mean,std=d[feat_name].mean(),d[feat_name].std()
  return (1 / (np.sqrt(2 * np.pi) * std)) *  np.exp(-((feat_value-mean)**2 / (2 * std**2 )))

def naive_bayes(train,test_values,decision_column):
  features=extract_feature()
  prior=prior_probablity_calculation(decision_column)
  for feature_name in features:
    if(is_feature_categorical(feature_name)):
      tables[feature_name]=table(train[feature_name],decision_column)
      
  Y_pred=[]
  for test_value in test_values:
    prob_feat_given_decision=[1]*len(prior)
    post_prob=[1]*len(prior)
    for i in range(len(prior)):
      for j in range(len(features)):
        if(is_feature_categorical(features[j])):
          if(tables[features[j]].get(test_value[j])!=None):
            prob_feat_given_decision[i] *=(tables[features[j]][test_value[j]][i])
          else:
            prob_feat_given_decision[i] *=0 
        else:
          prob_feat_given_decision[i] *=calculate_probablity(features[j],test_value[j],i,decision_column)
      post_prob[i]=prob_feat_given_decision[i]*prior[i]
    Y_pred.append(np.argmax(post_prob))
  return Y_pred

In [5]:
train, test = train_test_split(data, test_size=.20, random_state=10)
X_test=test.iloc[:,:-1].values
Y_actual=test.iloc[:,-1].values
Y_pred=naive_bayes(train,test_values=X_test,decision_column=train[decision_column_name])


print(confusion_matrix(Y_actual, Y_pred))
print(accuracy_score(Y_actual, Y_pred))

[[4 1]
 [3 8]]
0.75
